In [196]:
import gurobipy as gp
from gurobipy import GRB
import tomllib as tml
import numpy as np

Options looks like:
```
options = {
    "WLSACCESSID": "********-****-****-****-************",
    "WLSSECRET": "********-****-****-****-************",
    "LICENSEID": _____,
}
```

In [197]:
# get gurobi credentials
options = tml.load(open("license.toml", "rb"))

In [198]:
# establish env (must close)
env = gp.Env(params=options)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2527858
Academic license 2527858 - for non-commercial use only - registered to mb___@ur.rochester.edu


## Test with MPS File
I made a MPS file by solving LP.mod (written by Quan Luu) with GLPK for Windows.

In [199]:
m = gp.read("model.mps", env=env)
m.reset()
m.optimize()

Read MPS format model from file model.mps
Reading time = 0.01 seconds
LP: 757 rows, 729 columns, 2160 nonzeros
Discarded solution information
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2527858 - for non-commercial use only - registered to mb___@ur.rochester.edu
Optimize a model with 757 rows, 729 columns and 2160 nonzeros
Model fingerprint: 0xcc557f7f
Variable types: 0 continuous, 729 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-03, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective -0.0927000
Presolve removed 27 rows and 0 columns
Presolve time: 0.01s
Presolved: 730 rows, 729 columns, 2160 nonzeros
Variable types: 0 continuous, 729 integer (729

I'm not sure this tells us much. Check `glpk_out.txt`, it has the full output of this solution. 
Notable slice:
```
730 rows, 729 columns, 2160 non-zeros
      0: obj =  -4.657500000e-01 inf =   1.000e+01 (2)
      5: obj =  -1.523000000e-01 inf =   0.000e+00 (0)
*   224: obj =   6.790500000e-01 inf =   2.065e-14 (0) 1
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+   224: mip =     not found yet <=              +inf        (1; 0)
+   224: >>>>>   6.790500000e-01 <=   6.790500000e-01   0.0% (1; 0)
+   224: mip =   6.790500000e-01 <=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUND
Time used:   0.0 secs
Memory used: 1.9 Mb (1980226 bytes)
STATES:
[1 2 3]   [10 11 12]   [19 20 21]
[4 5 6] , [13 14 15] , [22 23 24].
[7 8 9]   [16 17 18]   [25 26 27]

BUCKETS:
Bucket 5: 1 2 3 4 5 6 7 8 9
Bucket 11: 11 12 19 21
Bucket 13: 10 13
Bucket 14: 14 15
Bucket 17: 16 17 18 25 26 27
Bucket 23: 20 22 23 24
```

## Problem Setup
I'm going to try and convert this outright to a Gurobi model.

In [200]:
# establish model (must close)
model = gp.Model(env=env)

# multiple solutions
model.Params.PoolSolutions = 10
model.Params.PoolSearchMode = 2

Set parameter PoolSearchMode to value 2


Generating the probability matrix.

In [201]:
def normpdf(x: float, mean: float, std: float) -> float:
  var = float(std)**2
  denom = (2*np.pi*var)**.5
  num = np.exp(-(float(x)-float(mean))**2/(2*var))
  return num/denom

def gen_state_prob(num_traits: int, num_states: int):
  mean = (num_states-1) / 2
  std = mean / 1.25

  state_prob = np.zeros(tuple([num_states] * num_traits), dtype=np.float64)
  for inds in np.ndindex(state_prob.shape):
    prob = 1
    for ind in inds:
      prob *= normpdf(ind, mean, std)
    
    state_prob[inds] = prob

  state_prob = state_prob / np.sum(state_prob)

  return state_prob.flatten()

Generating the reward matrix.

In [202]:
def unnumerize(num_traits: int, num_states: int, action: int):
  ufaction = []
  while action > 0:
    ufaction.insert(0, action % num_states)
    action = action // num_states

  while len(ufaction) < num_traits:
    ufaction.insert(0, 0)

  return ufaction

def reward_fn(param: tuple[float, float], state, action):
  
  l1dist = 0
  for s, a in zip(state, action):
    l1dist += abs(s - a)

  return param[0] - param[1] * l1dist

def reward_matrix(num_traits: int, num_states: int, reward_param: tuple[float, float]):
  total_states = num_states**num_traits
  res = np.array([[0 for _ in range(total_states)] for _ in range(total_states)], dtype=np.float64)
  for x in range(total_states):
    for y in range(total_states):
      s1 = unnumerize(num_traits, num_states, x)
      s2 = unnumerize(num_traits, num_states, y)

      res[x, y] = reward_fn(reward_param, s1, s2)

  return res

In [203]:
# parameters
t = 3
n_per_t = 3
n = n_per_t**t
k = 6
reward_param = (1, 0.5)

V = np.asarray([i for i in range(n)])

prob = gen_state_prob(t, n_per_t)

reward = reward_matrix(t, n_per_t, reward_param)

In [204]:
# Create Hess Variables
# GPL: var x{V, V} >= 0, <= 1, binary;
x = model.addVars(V, V, ub=1, vtype=GRB.BINARY)

In [205]:
# state objective
# GPL: maximize EP: sum{i in V} PROB[i] * sum{j in V} x[i, j] * REWARD[i, j];
# gp.quicksum( prob[i] * x[i][j] * reward[i][j] for i in V for j in V )
objective = gp.quicksum( gp.quicksum( (prob[i] * x[i,j] * reward[i][j]) for j in V) for i in V )
model.setObjective(objective, GRB.MAXIMIZE)

In [206]:
# add constraints

# /* there are exactly k buckets */
# kBucketConstr: sum{j in V} x[j, j] = k;
k_bucket = gp.quicksum( (x[j,j]) for j in V ) == k
model.addConstr(k_bucket)

# /* a state can only belong to one bucket */
# uniqueBucketConstr{i in V}: sum{j in V} x[i, j] = 1;
unique_bucket = (gp.quicksum( (x[i,j]) for j in V ) == 1 for i in V)
model.addConstrs(unique_bucket)

# /* a state cannot belong to a non-existant bucket */
# nonexBucketConstr{i in V, j in V}: x[i, j] <= x[j, j];
nonex_bucket = ( (x[i,j] <= x[j,j]) for i in V for j in V )
model.addConstrs(nonex_bucket)

model.update()

In [207]:
# can we solve?
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2527858 - for non-commercial use only - registered to mb___@ur.rochester.edu
Optimize a model with 757 rows, 729 columns and 2160 nonzeros
Model fingerprint: 0x43a585c0
Variable types: 0 continuous, 729 integer (729 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-03, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective -0.0925824
Presolve removed 27 rows and 0 columns
Presolve time: 0.01s
Presolved: 730 rows, 729 columns, 2160 nonzeros
Variable types: 0 continuous, 729 integer (729 binary)
Found heuristic solution: objective 0.3947496

Root relaxation: objective 6.793317e-01, 88 iterations, 0.00 seconds (0.00 work unit

### Solution Extraction
This was a little easier than I thought, thanks to Quan's code.

In [208]:
centers = [j for j in V if x[j,j].getAttr("x") == 1]

for j in centers:
    print(f"Bucket {j+1}: ", end="")
    members = [i for i in V if x[i,j].getAttr("x") == 1]
    for i in members:
        print(f"{i+1} ", end="")
    print()

Bucket 5: 1 2 3 5 7 9 
Bucket 13: 4 10 13 25 
Bucket 14: 11 14 
Bucket 15: 6 12 15 27 
Bucket 17: 8 16 17 18 26 
Bucket 23: 19 20 21 22 23 24 


```
STATES:
[1 2 3]   [10 11 12]   [19 20 21]
[4 5 6] , [13 14 15] , [22 23 24].
[7 8 9]   [16 17 18]   [25 26 27]

BUCKETS:
Bucket 5: 1 2 3 4 5 6 7 8 9
Bucket 11: 11 12 19 21
Bucket 13: 10 13
Bucket 14: 14 15
Bucket 17: 16 17 18 25 26 27
Bucket 23: 20 22 23 24
```

GLPK Output again for comparison.

In [209]:
# n solutions
n_solutions = model.getAttr("SolCount")

for n in range(0, n_solutions):
    model.params.SolutionNumber = n
    print(f"Solution {n}")
    centers = [j for j in V if x[j,j].getAttr("Xn") == 1]
    for j in centers:
        print(f"Bucket {j+1}: ", end="")
        members = [i for i in V if x[i,j].getAttr("Xn") == 1]
        for i in members:
            print(f"{i+1} ", end="")
        print()
    print()

    print("STATES:")
    print("[1 2 3]   [10 11 12]   [19 20 21]")
    print("[4 5 6] , [13 14 15] , [22 23 24].")
    print("[7 8 9]   [16 17 18]   [25 26 27]")
    print()

Solution 0
Bucket 5: 1 2 3 5 7 9 
Bucket 13: 4 10 13 25 
Bucket 14: 11 14 
Bucket 15: 6 12 15 27 
Bucket 17: 8 16 17 18 26 
Bucket 23: 19 20 21 22 23 24 

STATES:
[1 2 3]   [10 11 12]   [19 20 21]
[4 5 6] , [13 14 15] , [22 23 24].
[7 8 9]   [16 17 18]   [25 26 27]

Solution 1
Bucket 5: 2 3 5 7 9 
Bucket 13: 1 4 10 13 25 
Bucket 14: 11 14 
Bucket 15: 6 12 15 27 
Bucket 17: 8 16 17 18 26 
Bucket 23: 19 20 21 22 23 24 

STATES:
[1 2 3]   [10 11 12]   [19 20 21]
[4 5 6] , [13 14 15] , [22 23 24].
[7 8 9]   [16 17 18]   [25 26 27]

Solution 2
Bucket 5: 1 3 5 6 7 9 
Bucket 11: 2 11 12 21 
Bucket 13: 4 10 13 25 
Bucket 14: 14 15 
Bucket 17: 8 16 17 18 26 27 
Bucket 23: 19 20 22 23 24 

STATES:
[1 2 3]   [10 11 12]   [19 20 21]
[4 5 6] , [13 14 15] , [22 23 24].
[7 8 9]   [16 17 18]   [25 26 27]

Solution 3
Bucket 5: 1 2 3 4 5 7 9 
Bucket 13: 10 13 25 
Bucket 14: 11 14 
Bucket 15: 6 12 15 27 
Bucket 17: 8 16 17 18 26 
Bucket 23: 19 20 21 22 23 24 

STATES:
[1 2 3]   [10 11 12]   [19 20 21]
[4

In [210]:
# closing these objects for best practice

model.close()
m.close()
env.close()